In [1]:
#leyendo texto 1 de prueba
with open('Academic/nf0069.txt') as f:
    contents=f.read()
    print(contents)

nf0069: when talking about the French gangster film very often and 
paradoxically in French one talks about the policier that's the name of the 
genre so when you you see the word policier er I-E police which relates to the 
police it doesn't necessarily mean that it's to do with a police film or police 
procedure as the you have in the American genres but it's co-, covers all er 
virtually all the films er to do with crime er pretty much from the nineteen-
fifties onwards so er a gangster film and a police film will both be called 
policier in French and sometimes you'll see the in slang this becomes polar er 
i'm sorry i hope it's not too awkward if i move er polar is P-O-L-A-R er is yep 
sf0070: sorry 
nf0069: okay we're just beginning polar is the slang word for policier so le 
polar refers to the genre of the police crime gangster movie in French er 
specifically from the nineteen-fifties late forties fifties onwards and so that 
i will 
talk today about a particular genre or subg

In [3]:
#dividir por salto de linea
contents= contents.split('\n')

In [4]:
len(contents)

578

In [5]:
#dividir en oraciones, importar la funcion
from nltk import sent_tokenize

#dividir en oraciones
sentences = []
for sentence in contents:
    sent_token = sent_tokenize(sentence)
    sentences+=[sent_token]

In [7]:
#aplanar la lista de strings
flat_sentences = []
for item in sentences:
        for word in item:
            flat_sentences+=[word]

In [9]:
#remover puntuacion
def remove_punctuation(text):
    """ Get rid of punctuation except apostrophes """
    return re.sub(r"[^\P{P}\']+", "", text)

In [10]:
flat_sentences_nopunc =[]

for punto in flat_sentences:
    flat_sentences_nopunc+=[remove_punctuation(flat_sentences)]

error: bad escape \P at position 2